# Advent of Code

## 2020-012-020
## 2020 020

https://adventofcode.com/2020/day/20

In [4]:
import math

def read_tiles(filename):
    with open(filename, 'r') as f:
        raw = f.read().strip().split('\n\n')
    
    tiles = {}
    for block in raw:
        lines = block.strip().split('\n')
        tile_id_line = lines[0]
        tile_id = int(tile_id_line.replace('Tile ', '').replace(':', ''))
        image = lines[1:]
        tiles[tile_id] = image
    return tiles

def rotate_tile(tile):
    # Rotate tile 90 degrees clockwise
    return list(map(''.join, zip(*tile[::-1])))

def flip_tile(tile):
    # Flip tile horizontally
    return tile[::-1]

def all_transformations(tile):
    # Generate all 8 transformations (4 rotations, and their flips)
    transformations = []
    current = tile
    for _ in range(4):
        transformations.append(current)
        transformations.append(flip_tile(current))
        current = rotate_tile(current)
    # Remove duplicates if any
    unique = []
    seen = set()
    for t in transformations:
        tuple_form = tuple(t)
        if tuple_form not in seen:
            seen.add(tuple_form)
            unique.append(t)
    return unique

def edges(tile):
    # Returns top, right, bottom, left edges of the tile
    top = tile[0]
    bottom = tile[-1]
    left = ''.join(row[0] for row in tile)
    right = ''.join(row[-1] for row in tile)
    return (top, right, bottom, left)

def fits(grid, x, y, placed_tiles, tile):
    # Check if tile fits into position (x, y) in the grid
    # grid size:
    N = int(math.sqrt(len(placed_tiles)))
    top_neighbor = grid[x-1][y] if x > 0 else None
    left_neighbor = grid[x][y-1] if y > 0 else None
    
    t, r, b, l = edges(tile)
    
    # Check top match
    if top_neighbor is not None:
        top_tile = placed_tiles[top_neighbor]
        _, _, top_bottom, _ = edges(top_tile)
        if top_bottom != t:
            return False

    # Check left match
    if left_neighbor is not None:
        left_tile = placed_tiles[left_neighbor]
        _, left_right, _, _ = edges(left_tile)
        if left_right != l:
            return False
    
    return True

def solve_puzzle(tiles):
    # We'll do a backtracking search.
    N = int(math.sqrt(len(tiles)))
    tile_ids = list(tiles.keys())
    transformations = {}
    for tid in tile_ids:
        transformations[tid] = all_transformations(tiles[tid])

    grid = [[None]*N for _ in range(N)]
    used = set()

    def backtrack(pos=0):
        if pos == N*N:
            return True
        x = pos // N
        y = pos % N
        for tid in tile_ids:
            if tid in used:
                continue
            for tvar in transformations[tid]:
                if fits(grid, x, y, {g: transformations_map[g] for g in used}, tvar):
                    grid[x][y] = tid
                    used.add(tid)
                    # Temporarily store chosen transformation
                    chosen_transformations[tid] = tvar
                    if backtrack(pos+1):
                        return True
                    used.remove(tid)
                    grid[x][y] = None
        return False

    # We need to store the final transformations to check edges later if needed
    chosen_transformations = {}
    # We'll create a helper to map tile to its chosen transformation
    transformations_map = chosen_transformations

    if backtrack():
        return grid, chosen_transformations
    else:
        raise ValueError("No solution found.")

def main():
    tiles = read_tiles('input.txt')
    grid, chosen = solve_puzzle(tiles)
    # Corner tiles:
    # Once arranged, the corners are:
    # Top-left: grid[0][0]
    # Top-right: grid[0][-1]
    # Bottom-left: grid[-1][0]
    # Bottom-right: grid[-1][-1]
    corners = [grid[0][0], grid[0][-1], grid[-1][0], grid[-1][-1]]
    product = 1
    for c in corners:
        product *= c
    print(product)

if __name__ == '__main__':
    main()

64802175715999


In [5]:
import math

def read_tiles(filename):
    with open(filename, 'r') as f:
        raw = f.read().strip().split('\n\n')
    tiles = {}
    for block in raw:
        lines = block.strip().split('\n')
        tile_id_line = lines[0]
        tile_id = int(tile_id_line.replace('Tile ', '').replace(':', ''))
        image = lines[1:]
        tiles[tile_id] = image
    return tiles

def rotate_tile(tile):
    # Rotate tile 90 degrees clockwise
    return list(map(''.join, zip(*tile[::-1])))

def flip_tile(tile):
    # Flip tile vertically
    return tile[::-1]

def all_transformations(tile):
    # Generate all 8 transformations (4 rotations, and flips)
    transformations = []
    current = tile
    for _ in range(4):
        transformations.append(current)
        transformations.append(flip_tile(current))
        current = rotate_tile(current)
    # Remove duplicates if any
    unique = []
    seen = set()
    for t in transformations:
        tuple_form = tuple(t)
        if tuple_form not in seen:
            seen.add(tuple_form)
            unique.append(t)
    return unique

def edges(tile):
    # Returns top, right, bottom, left edges of the tile
    top = tile[0]
    bottom = tile[-1]
    left = ''.join(row[0] for row in tile)
    right = ''.join(row[-1] for row in tile)
    return (top, right, bottom, left)

def fits(grid, x, y, placed_tiles, chosen_transformations, tid, tvar):
    # Check if tile fits into position (x, y) in the grid
    N = len(grid)
    t, r, b, l = edges(tvar)

    # Check top
    if x > 0 and grid[x-1][y] is not None:
        top_tid = grid[x-1][y]
        top_tile = chosen_transformations[top_tid]
        _, _, top_bottom, _ = edges(top_tile)
        if top_bottom != t:
            return False

    # Check left
    if y > 0 and grid[x][y-1] is not None:
        left_tid = grid[x][y-1]
        left_tile = chosen_transformations[left_tid]
        _, left_right, _, _ = edges(left_tile)
        if left_right != l:
            return False

    return True

def solve_puzzle(tiles):
    # We'll do a backtracking search to arrange tiles correctly.
    N = int(math.sqrt(len(tiles)))
    tile_ids = list(tiles.keys())
    transformations = {}
    for tid in tile_ids:
        transformations[tid] = all_transformations(tiles[tid])

    grid = [[None]*N for _ in range(N)]
    used = set()
    chosen_transformations = {}

    def backtrack(pos=0):
        if pos == N*N:
            return True
        x = pos // N
        y = pos % N
        for tid in tile_ids:
            if tid in used:
                continue
            for tvar in transformations[tid]:
                if fits(grid, x, y, used, chosen_transformations, tid, tvar):
                    grid[x][y] = tid
                    used.add(tid)
                    chosen_transformations[tid] = tvar
                    if backtrack(pos+1):
                        return True
                    used.remove(tid)
                    grid[x][y] = None
        return False

    if backtrack():
        return grid, chosen_transformations
    else:
        raise ValueError("No solution found.")

def remove_borders(tile):
    # Remove the border from a single tile
    return [row[1:-1] for row in tile[1:-1]]

def assemble_full_image(grid, chosen):
    # After solving the puzzle, remove borders and stitch together
    N = len(grid)
    # All tiles are now correctly oriented
    stripped_tiles = [[remove_borders(chosen[grid[x][y]]) for y in range(N)] for x in range(N)]

    # Each stripped tile is now (tile_size-2) in both dimensions
    tile_size = len(stripped_tiles[0][0])
    full_image = []
    for x in range(N):
        for row_i in range(tile_size):
            row_parts = []
            for y in range(N):
                row_parts.append(stripped_tiles[x][y][row_i])
            full_image.append(''.join(row_parts))
    return full_image

def rotate_image(image):
    return list(map(''.join, zip(*image[::-1])))

def flip_image(image):
    return image[::-1]

def all_image_transformations(image):
    transformations = []
    current = image
    for _ in range(4):
        transformations.append(current)
        transformations.append(flip_image(current))
        current = rotate_image(current)
    unique = []
    seen = set()
    for t in transformations:
        tup = tuple(t)
        if tup not in seen:
            seen.add(tup)
            unique.append(t)
    return unique

# Sea monster pattern (relative coordinates of '#')
# Using the provided pattern:
#                   #
# #    ##    ##    ###
#  #  #  #  #  #  #
#
# Let's encode the sea monster as a list of (x,y) offsets
sea_monster = [
    (0,18),
    (1,0),(1,5),(1,6),(1,11),(1,12),(1,17),(1,18),(1,19),
    (2,1),(2,4),(2,7),(2,10),(2,13),(2,16)
]

def count_sea_monsters(image):
    max_x = len(image)
    max_y = len(image[0])
    count = 0
    for x in range(max_x - 2):
        for y in range(max_y - 19):
            if all(image[x+dx][y+dy] == '#' for dx, dy in sea_monster):
                count += 1
    return count

def main():
    tiles = read_tiles('input.txt')
    grid, chosen = solve_puzzle(tiles)
    full_image = assemble_full_image(grid, chosen)

    # Count total '#' in the full image
    total_hash = sum(row.count('#') for row in full_image)

    # Try all transformations to find sea monsters
    best_monsters = 0
    for img in all_image_transformations(full_image):
        monsters = count_sea_monsters(img)
        if monsters > best_monsters:
            best_monsters = monsters

    # Each sea monster has 15 '#' (as calculated previously)
    sea_monster_hash = 15
    roughness = total_hash - best_monsters * sea_monster_hash
    print(roughness)

if __name__ == '__main__':
    main()

2146
